In [1]:
!pip3 install keras
!pip3 install tensorflow
!pip3 install keras
!pip3 install numpy
!pip3 install sklearn
!pip3 install pandas
!pip3 install opencv-python

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.models import model_from_json
from tensorflow.keras.utils import to_categorical

num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

df = pd.read_csv('./fer2013.csv')
df = df.drop(columns=['Usage'])

y = df['emotion'].values

X = []

datapoints = df['pixels'].tolist()

for xseq in datapoints:
    xx = [int(xp)/255 for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)   

y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# create model
model = Sequential()

# add model layers
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True)

Epoch 1/100
449/449 [==============================] - 334s 743ms/step - loss: 1.9987 - accuracy: 0.2084
Epoch 2/100
449/449 [==============================] - 335s 746ms/step - loss: 1.8359 - accuracy: 0.2431
Epoch 3/100
449/449 [==============================] - 336s 749ms/step - loss: 1.8078 - accuracy: 0.2569
Epoch 4/100
449/449 [==============================] - 337s 750ms/step - loss: 1.7337 - accuracy: 0.2967
Epoch 5/100
449/449 [==============================] - 337s 752ms/step - loss: 1.6150 - accuracy: 0.3560
Epoch 6/100
449/449 [==============================] - 336s 749ms/step - loss: 1.5274 - accuracy: 0.4013
Epoch 7/100
449/449 [==============================] - 337s 750ms/step - loss: 1.4760 - accuracy: 0.4193
Epoch 8/100
449/449 [==============================] - 337s 750ms/step - loss: 1.4302 - accuracy: 0.4379
Epoch 9/100
449/449 [==============================] - 337s 750ms/step - loss: 1.3983 - accuracy: 0.4531
Epoch 10/100
449/449 [==============================] -

449/449 [==============================] - 350s 779ms/step - loss: 0.4382 - accuracy: 0.8549
Epoch 79/100
449/449 [==============================] - 349s 777ms/step - loss: 0.4260 - accuracy: 0.8587
Epoch 80/100
449/449 [==============================] - 344s 765ms/step - loss: 0.4248 - accuracy: 0.8613
Epoch 81/100
449/449 [==============================] - 345s 768ms/step - loss: 0.4275 - accuracy: 0.8600
Epoch 82/100
449/449 [==============================] - 346s 771ms/step - loss: 0.4109 - accuracy: 0.8621
Epoch 83/100
449/449 [==============================] - 346s 770ms/step - loss: 0.4042 - accuracy: 0.8690
Epoch 84/100
449/449 [==============================] - 346s 771ms/step - loss: 0.4042 - accuracy: 0.8671
Epoch 85/100
449/449 [==============================] - 347s 772ms/step - loss: 0.4041 - accuracy: 0.8680
Epoch 86/100
449/449 [==============================] - 347s 773ms/step - loss: 0.3962 - accuracy: 0.8721
Epoch 87/100
449/449 [==============================] - 346

In [19]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.55      0.54      1005
           1       0.79      0.50      0.61       101
           2       0.49      0.47      0.48      1006
           3       0.82      0.86      0.84      1802
           4       0.57      0.48      0.52      1222
           5       0.80      0.75      0.77       804
           6       0.54      0.64      0.59      1238

    accuracy                           0.64      7178
   macro avg       0.65      0.61      0.62      7178
weighted avg       0.64      0.64      0.64      7178

